In [1]:
import logging
import sys
import nest_asyncio
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    Response,
    
)
import pinecone
import openai
import streamlit as st
from llama_index.vector_stores import PineconeVectorStore
from llama_index.llms import OpenAI
from llama_index.evaluation import CorrectnessEvaluator

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

nest_asyncio.apply()

/Users/fran/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/fran/Library/Python/3.9/lib/python/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
openai.api_key = st.secrets.openai_key
api_key = "2bcecfc8-35f9-44dc-9459-ab0cc2781b29"
pinecone.init(api_key=api_key, environment="gcp-starter")
pinecone_index = pinecone.Index("tailwind-hugging")

In [3]:
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0.5, system_prompt="You are an expert on TailwindCSS and your job is to answer technical questions. Assume that all questions are related to TailwindCSS. Keep your answers technical and based on facts – do not hallucinate features."))
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)

In [4]:
evaluator = CorrectnessEvaluator(service_context=service_context)

In [21]:
chat_engine = index.as_chat_engine(chat_mode="react", verbose=True)
query = query_str = "What is border-radius? And how to use it inTailwindCSS?"
response = chat_engine.chat(query)
reference = "It is a property that allows you to round the corners of an element. You can use it in TailwindCSS by adding the rounded class to an element. For example, rounded-lg will round the corners of an element by 0.5rem."

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Thought: I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'What is border-radius?'}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Observation: Border-radius is a CSS property that is used to control the roundedness of the corners of an element. It allows you to apply rounded corners to elements such as buttons, divs, or other HTML elements. The roundedness can be applied to all corners of an element or to individual corners separately. The size of the rounded corners can be customized u

In [16]:
result = evaluator.evaluate(
    query=query,
    response=response.response,
    reference=reference,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [22]:
import pandas as pd
def display_eval_df(response: Response, eval_result: str) -> None:
    if response.source_nodes == []:
        print("no response!")
        return
    score = "{:.1f}".format(eval_result.score)
    eval_df = pd.DataFrame(
        {
            "Response": str(response),
            "Reference": reference,
            "Score": score,
            "Feedback": eval_result.feedback,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Reference","Feedback"]
    )
    display(eval_df)

In [23]:
display_eval_df(response, result);

,Response,Reference,Score,Feedback
0,"Border-radius is a CSS property that is used to control the roundedness of the corners of an element. It allows you to apply rounded corners to elements such as buttons, divs, or other HTML elements. The roundedness can be applied to all corners of an element or to individual corners separately. The size of the rounded corners can be customized using predefined values or by specifying custom values. In TailwindCSS, you can use the `rounded-*` utilities to apply border-radius to elements. These utilities allow you to specify different levels of border-radius, such as `rounded-lg` for a larger border-radius or `rounded-none` for no border-radius. Additionally, you can customize the border-radius values by editing the `theme.borderRadius` section in your Tailwind config file.","It is a property that allows you to round the corners of an element. You can use it in TailwindCSS by adding the rounded class to an element. For example, rounded-lg will round the corners of an element by 0.5rem.",4.0,"The generated answer is relevant and provides accurate information about when to choose `display: inline-block` and `display: block`. However, the reference answer is more concise and provides a clearer explanation of the specific scenarios where each property is most beneficial."
